> Check if the required packages are installed

```python
cd aiffel 
git clone https://github.com/airobotlab/KoChatGPT  
cd KoChatGPT/colossalai_ChatGPT_230319/
pip install .  
```

In [1]:
import torch

print("Torch version:{}".format(torch.__version__)) # Torch version:1.12.1
print("Cuda version: {}".format(torch.version.cuda)) # Cuda version: 11.3
!pip list | grep transformers # transformers 4.28.0

Torch version:1.12.1
Cuda version: 11.3
transformers                  4.28.0


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import numpy

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


- Check tokenizer 

In [3]:
tokenizer.max_model_input_sizes

{'gpt2': 1024,
 'gpt2-medium': 1024,
 'gpt2-large': 1024,
 'gpt2-xl': 1024,
 'distilgpt2': 1024}

- 어떻게 토크나이징 할까요?

In [4]:
input_txt = "바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까."

In [5]:
tokens = tokenizer(input_txt).tokens()

In [8]:
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].numpy()

In [6]:
pd.options.display.max_columns = 40
pd.options.display.max_rows = 60

In [9]:
df = pd.DataFrame([tokens, input_ids[0]], index=["kogpt-2_tokens", "Input_IDs"])
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
kogpt-2_tokens,▁바람,도,▁없는,▁공중에,▁수직,의,▁파,문을,▁내,이며,▁고,요,히,▁떨어지는,▁오동,잎은,▁누,구의,▁발자,취,▁입,니까,.
Input_IDs,10891,7235,9712,49207,14438,8143,9203,9941,9094,9639,9065,8084,8811,21215,34769,19985,9669,10139,21626,8408,9241,23775,389


## About Decoder

> Greedy Search Decoder

In [10]:
max_length=128
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=max_length, do_sample=False)
print(tokenizer.decode(output_greedy[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까.'
"그렇다면 그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리


시퀀스가 반복되어 출력되는군요.
그리디 서치 디코딩시 발견되는 전형적인 현상입니다.

이번엔 빔 서치 디코딩을 사용하고 n-gram 패널티까지 부과해보겠습니다.

In [11]:
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)

In [12]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=10, no_repeat_ngram_size=2,
                             do_sample=False)
print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까.'
"그렇지 않습니다."
"어떻게 된 일입니까?"
그녀는 고개를 갸웃거렸다.
"아니, 그게 무슨 말씀이신지 모르겠습니다만."
"무슨 말씀인지 알 수가 없군요."
아무런 대답도 하지 않은 채 그녀는 고개를 끄덕였다.
"그래, 알았어."
그녀의 눈에서 눈물이 주르륵 흘러내렸다.
그녀가 다시 입을 열었다.
"정말 죄송합니다, 고마워요, 고맙습니다"
"


입력 시퀀스와 별 상관 없어 보이는 긴 문단이 생성됩니다.
그럼에도 생성된 문단은 제법 맥락을 갖춘 듯 보입니다.
하지만 문장 간의 정합성이나 일관성은 다소 떨어지는 부분도 관찰됩니다.
이번엔 샘플링 기법까지 추가해 보겠습니다.

> plus 샘플링 기법

In [13]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=7, no_repeat_ngram_size=2,
                             do_sample=True, temperature=2.0, top_k=50)
print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까."
"그럼 어디에서 그런 소리를 내는 걸까요?"
소녀는 고개를 흔들었다.
"저도 모르는 거죠. 이게 어떤 소리인가 하는 생각도 해 보지는 못했지만."
그렇게 대답한 소녀는 대답하지 않았다.
하지만 그녀는 소파의 눈을 똑바로 보았다.
"아무도 못 알아들었잖아요. 저도 모르지 않습니까."
"아, 아뇨. 알고 싶지는 않습니다. 제가 잘 모르고 있긴 한데. 그런데 그 소리만 들리는 건 아니에요. 그런 것 같던데."



> Top_p sampling method

In [14]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=7, no_repeat_ngram_size=2,
                             do_sample=True, top_p=0.90)
print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까."
"그렇지. 그건 그렇고."
"아니, 그게 무슨 소리예요?"
그녀는 한숨을 내쉬었다.
"어떻게 된 일인지 모르겠어요."
어느새 그녀는 발걸음을 멈췄다.
"무슨 소리야? 도대체 무슨 짓을 하고 있는 거야!"
순간 그녀의 눈에서 눈물이 주르르 흘러내렸다.
"도대체 뭐라고 말해야 할지 모르겠군. 뭔가 이상하군."
그녀의 눈동자가 하얗게


> SFT 시도할 initial model dataset

In [15]:
import json 
data_path_1_SFT = '/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl' 

In [16]:
with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

In [17]:
print(len(list_data_dict))
list_data_dict[:3]

12000


[{'prompt': '불고기용 고기 한우에요?',
  'completion': "'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.",
  'tokens': 193},
 {'prompt': '쓰던 앱이 유료로 전환됐어',
  'completion': "'어떤 앱인지 모르기 때문에 정확한 답변을 드리기 어렵습니다. 하지만, 일반적으로 유료 전환된 앱은 기존 무료 앱에서 추가적인 기능이나 서비스를 제공하는 경우가 많습니다. 이 경우, 유료 전환 이전 가격이 매우 저렴하거나 무료인 경우가 많으므로, 앱을 계속 사용하려면 추가적인 비용을 지불해야 할 수 있습니다. 하지만, 유료 전환 이후에는 기존 사용자 또는 새로운 사용자 모두가 추가 기능과 높은 품질의 서비스를 이용할 수 있게 됩니다. 따라서, 앱 개발자는 유료 모델로 전환함으로써 앱의 수익을 증가시키고 더 나은 서비스를 제공할 수 있습니다.",
  'tokens': 288},
 {'prompt': '여친이랑 다툼',
  'completion': "'저는 인공지능이기 때문에 여친과의 다툼에 대해서는 조언을 드릴 수는 없습니다. 하지만 다투게 되면 상대방의 말을 경청하는 태도와 서로의 감정을 존중하는 것이 중요합니다. 논리적이고 차분한 대화를 통해 문제를 해결해보세요. 그리고 서로를 이해하고 서로의 의견을 수용하는 것이 중요합니다.",
  'tokens': 153}]

# RM

In [18]:
data_path_2_RM = '/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl'
with open(data_path_2_RM, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

10220


[{'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?',
  'completion_0': 'Allow me to answer your question. I know that you are curious about me.',
  'completion_1': '번디는 다양한 인터뷰자들과 뉴스홍보 담당자들과의 면담 때 밝혔다.',
  'completion_2': '라이언에게 말했다.',
  'ranking': [2, 1, 0]},
 {'prompt': '개포주공아파트는 몇 단지로 이루어져 있나?',
  'completion_0': '개포주공아파트는 다섯 단지로 이루어져 있습니다.',
  'completion_1': '이날 목송에서 구글상위노',
  'completion_2': '개포주공아파트는 총 27개 단지로 이루어져 있습니다.',
  'ranking': [2, 0, 1]},
 {'prompt': '김영삼의 후보 시절 지역표심을 겨냥한 발언을 문제삼은 후보는?',
  'completion_0': 'The diameter of the Metallic domain is bigger than the Hyperonic domain.',
  'completion_1': '이 질문은 조금 불분명합니다. 김영삼 대통령이 후보 시절에 어떤 발언을 했고, 누가 그 발언을 문제삼았는지에 따라 답이 다를 수 있습니다.\\n\\n만약 김영삼 대통령이 후보 시절에 지역표심을 겨냥한 발언을 했다는 가정하에, 그 발언을 문제삼은 후보가 누구였는지를 대답하자면, 그 답은 이화선 당시 민주당 대통령 후보가 될 것입니다. 1992년 총선 때, 김영삼 대선후보는 "집값이 오른 노량진역 부근의 부동산 가격은 세월호 폭침 후 \\\'강남 도시재생\\\' 일환으로 상승했다"는 발언을 했습니다. 하지만 이화선 후보는 이 발언을 "전국적으로 경제적 발전이 이루어지지 않은 지방민의 마음을 멀리해지려는 무례한 발언"이라고 비판하며 문

> What is Reward Model

Reward model evaluates the quality of the model’s responses based on human feedback. It is trained using a technique called reinforcement learning from human feedback (RLHF), which involves rewarding the model for providing useful answers and discouraging inappropriate answers. A prompt is a text input that is given to the GPT model to generate a response. A completion is a text output that is produced by the GPT model based on the prompt. A ranking is a numerical score that is assigned to the completion by the reward model based on how useful, relevant, and appropriate it is. The reward model generates the ranking by taking a series of prompts and completions as input and outputting a scalar value reward. The reward is higher for completions that are more informative, coherent, engaging, and polite, and lower for completions that are less so.  The reward model also penalizes completions that are off-topic, repetitive, nonsensical, or offensive. The reward model is updated continuously as it receives more human feedback.

> Relevance to KoGPT dataset

The prompt is a question from the ELI5 subreddit, which is a platform where people can ask and answer questions in a simple and easy way.

The completion is an answer generated by a pre-trained language model, such as GPT-3.5 or GPT-2, given the prompt as input.

The ranking is a list of three numbers, indicating the relative quality of the three completions from the perspective of human labelers. The numbers range from 0 to 2, where 0 is the worst and 2 is the best. For example, [2, 1, 0] means that the first completion is the best, the second completion is the second best, and the third completion is the worst.

The logic behind this dataset is to train a reward model that can mimic human preferences and judgments on the quality of text generated by language models. The reward model can then be used to fine-tune the language models using reinforcement learning, so that they can generate better answers for the given questions.

- Reference 
    - https://www.unite.ai/what-is-reinforcement-learning-from-human-feedback-rlhf/

# PPO
- Dataset

In [19]:
data_path_3_PPO = '/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl'
with open(data_path_3_PPO, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

12000


[{'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?'},
 {'prompt': '개포주공아파트는 몇 단지로 이루어져 있나?'},
 {'prompt': '김영삼의 후보 시절 지역표심을 겨냥한 발언을 문제삼은 후보는?'}]

PPO is a reinforcement learning algorithm that is commonly used for training large language models like ChatGPT and Instruct GPT. It works by adjusting the parameters of the model such that the reward is maximized for each generated response. PPO integrates a per-token Kullback-Leibler (KL) penalty from the SFT model.
The KL divergence measures the similarity between two distribution functions and penalizes extreme distances. 
PPO is based on the idea of trust regions, which are regions in the parameter space where the model’s behavior does not change too much. PPO tries to find the optimal policy within these regions by using a surrogate objective function that approximates the true objective function. PPO also uses a clipping mechanism that prevents the model from taking too large or too small steps in the parameter space. This way, PPO balances exploration and exploitation and avoids instability and degradation of performance .
The prompts are questions that are collected from a Korean question dataset and converted to a jsonl format. The prompts are designed to elicit informative, coherent, engaging, and polite responses from the ChatGPT-replica model.

# 26-3. Supervised Fine-Tuning

## SFT
이번 스텝에서는 kogpt-2를 instruction dataset으로 SFT를 진행해 보겠습니다.

먼저 필요한 라이브러리들을 불러오겠습니다.

In [20]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim import Adam
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments
from copy import deepcopy
import copy
import logging
import json
from dataclasses import dataclass

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


AutoModelForCausalLM.from_pretrained -> AutoTokenizer.from_pretrained

In [21]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

In [22]:
print(tokenizer)

GPT2TokenizerFast(name_or_path='skt/kogpt2-base-v2', vocab_size=51200, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=True)


In [23]:
from typing import Optional, Dict, Sequence

In [24]:
class SFT_dataset(Dataset):

    def __init__(self, data_path_1_SFT: str, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")

        pattern_instruction = 'prompt'  # instruction
        pattern_output = 'completion'  # response

        data_path_1_SFT = '/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl'
        with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
            list_data_dict = json.load(json_file)

        PROMPT_DICT = {
            "prompt_input": (
                "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
            )
        }

        prompt_input = PROMPT_DICT["prompt_input"]

        sources = []
        for example in list_data_dict:
            tmp = prompt_input.format_map(example)
            sources.append(tmp)

        targets = []
        for example in list_data_dict:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")
        examples = [s + t for s, t in zip(sources, targets)]

        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target

        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = -100

        data_dict = dict(input_ids=input_ids, labels=labels)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))


    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )


    def __len__(self):
        return len(self.input_ids)


    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

Question 9) 
SFT_dataset 클래스의 이니셜라이저에서
label[:source_len] = -100 코드의 -100이 의미하는 게 무엇인가요?
해당 코드가 필요한 이유와 그 기능은 무엇인가요?

A) 
-  -100은 Pytorch에서 특정 토큰의 손실 계산을 무시하도록 지시하는데 사용되는 숫자이다.
- 필요한 이유: 언어 생성 모델은 입력 시퀀스를 받아 다음 토큰을 예측하도록 훈련되지만 입력 시퀀스는 토큰들의 예측 대상이 아니므로 무시해야 한다.
- 기능: label 배열의 처음부터 source_len까지의 토큰들을 -100으로 설정을 해서 모델은 입력 시퀀스의 토큰들에 대해서는 손실을 계산하지 않고 출력 시퀀스의 토큰들에 대해서만 손실을 계산한다.
- 참고 링크: [﻿pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) 

In [25]:
@dataclass
class DataCollatorForSupervisedDataset(object): 

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value= -100)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

Q10. DataCollatorForSupervisedDataset 클래스에서
labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value= -100) 코드의
padding_value= -100 인자의 -100은 어떤 기능을 하나요?

- 레이블의 패딩된 부분이 모델 학습에 영향을 미치지 않도록 한다.


> 이제 SFT_dataset 클래스를 사용해 훈련셋을 만들고 data collator 인스턴스를 만들겠습니다.

In [26]:
train_dataset = SFT_dataset(data_path_1_SFT='/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl', tokenizer=tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

In [27]:
print('input : %s'%train_dataset.input_ids[0])
print('output: %s'%train_dataset.labels[0])

input : tensor([  739,   378,   378,   378, 14659, 13394, 37091, 10651,   383, 25841,
         8006, 14914,   375,  7673, 20479,  8091, 22311,  9036, 30902, 13675,
          375,   378,   378,   378, 41951,   454,  9549, 20549,   383,  8142,
         7192, 14914,   382, 37767, 13753,  8263,  7166,   739,  8352,  7659,
         9594, 25585, 13600,  8022,  9378, 11532,  9887, 11218,  9111, 16691,
        10351, 10561,  9128, 20479,  8091,  9065,  9446,  9036, 28420, 26521,
        10163, 26367,  6958,  9030,  9882, 12317, 25882,  9209, 37194, 10351,
         9036, 12168, 10529, 15989,  9719, 15434, 10552, 11188, 13362,  9036,
        15805, 11300, 11846,  9146, 16691,  9181,  7397, 15806, 13480, 11342,
        17596,  9161, 19996,  9025, 25006, 18595,  9966, 12592, 10751, 11814,
         8711,  9046, 12450,  9117,  7377, 12521,     1])
output: tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -10

Q11. 아래 코드블럭에 train_dataset.input_ids[0] 와 train_dataset.labels[0] 를
디코딩하는 함수를 작성하여 정수토큰들의 인코딩전 원래 문장이
어떤 형태로 토크나이징 되었는지 확인해보세요.

ㅎㅇ 여기서부터 대충 (20231207 11:21 AM) 
26-5 까지 가야해..

In [ ]:
# 디코딩 함수를 작성해보세요.


# 26-5 Proximal Policy Optimization

In [28]:
from copy import deepcopy

import torch
from torch.optim import Adam
from chatgpt.models.base import RewardModel
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.trainer import PPOTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from transformers import AutoTokenizer

노드에서 소개하는 KoChatGPT의 경우
PPO에 사용할 actor모델은 1단계 SFT 모델을,
critic모델은 2단계 RM 모델을 사용합니다.

그리고 actor 모델이 critic 모델로부터 피드백을 받아 파라미터를 업데이트 할 때
적절한 페널티를 줄 수 있도록 하는 initial model은
SFT모델을 그대로 freezing 하여 사용합니다.

토크나이저는 pretrain 모델인 kogpt-2의 토크나이저를 그대로 사용해야겠죠?

In [29]:
with NaiveStrategy().model_init_context():
    actor = GPTActor(pretrained='/aiffel/KoChatGPT/output_1_SFT', lora_rank=0).to(torch.cuda.current_device())
    critic = GPTCritic(pretrained='aiffel/KoChatGPT/output_2_RM', lora_rank=0).to(torch.cuda.current_device())

    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", 
        model_max_length=512
    )

    initial_model = deepcopy(actor)
    reward_model = RewardModel(deepcopy(critic.model), deepcopy(critic.value_head)).to(torch.cuda.current_device())

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /opt/conda/lib/python3.9/site-packages/transformers/configuration_utils.py:628 in                │
│ _get_config_dict                                                                                 │
│                                                                                                  │
│   625 │   │   │                                                                                  │
│   626 │   │   │   try:                                                                           │
│   627 │   │   │   │   # Load from local folder or from cache or download from model Hub and ca   │
│ ❱ 628 │   │   │   │   resolved_config_file = cached_file(                                        │
│   629 │   │   │   │   │   pretrained_model_name_or_path,                                         │
│   630 │   │   │   │   │   configuration_file,                                                    │
│   631 │   │   │   │   │   cache_dir=cache_dir,                                                   │
│                                                                                                  │
│ /opt/conda/lib/python3.9/site-packages/transformers/utils/hub.py:409 in cached_file              │
│                                                                                                  │
│    406 │   user_agent = http_user_agent(user_agent)                                              │
│    407 │   try:                                                                                  │
│    408 │   │   # Load from URL or cache if already cached                                        │
│ ❱  409 │   │   resolved_file = hf_hub_download(                                                  │
│    410 │   │   │   path_or_repo_id,                                                              │
│    411 │   │   │   filename,                                                                     │
│    412 │   │   │   subfolder=None if len(subfolder) == 0 else subfolder,                         │
│                                                                                                  │
│ /opt/conda/lib/python3.9/site-packages/huggingface_hub/utils/_validators.py:110 in _inner_fn     │
│                                                                                                  │
│   107 │   │   │   kwargs.items(),  # Kwargs values                                               │
│   108 │   │   ):                                                                                 │
│   109 │   │   │   if arg_name in ["repo_id", "from_id", "to_id"]:                                │
│ ❱ 110 │   │   │   │   validate_repo_id(arg_value)                                                │
│   111 │   │   │                                                                                  │
│   112 │   │   │   elif arg_name == "token" and arg_value is not None:                            │
│   113 │   │   │   │   has_token = True                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.9/site-packages/huggingface_hub/utils/_validators.py:158 in               │
│ validate_repo_id                                                                                 │
│                                                                                                  │
│   155 │   │   raise HFValidationError(f"Repo id must be a string, not {type(repo_id)}: '{repo_   │
│   156 │                                                                                          │
│   157 │   if repo_id.count("/") > 1:                                                             │
│ ❱ 158 │   │   raise HFValidationError(                                                           │
│   159 │   │   │   "Repo id must be in the form 'repo_name' 

---- 

등 등 등

PPO는 별도의 PPOTrainer 클래스를 설계하여 학습시켜줘야 합니다.
빠르게 실습해보기 위해 1epoch만 돌려보겠습니다.

PPO에 대해 좀 더 자세히 알고 싶으신 분들은 허깅페이스에서 제공하는 Deep RL Course를 참고하세요.

이제 PPO 학습을 진행하도록 하겠습니다.